In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd

# import xgboost as xgb
# from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import lightgbm as lgb
import warnings
warnings.simplefilter("ignore")



tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

tf_df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    tf_df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(tf_df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)

kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [2]:
import json

list_bayesOpt = glob.glob('../models/SVD/LightGBM_BayesOpt_dec12/logs_shuffle*.json')

opt_best_df = pd.DataFrame()
for jsonFile in list_bayesOpt:
    with open(jsonFile) as f:
        optList = []
        for jsonObj in f:
            optDict = json.loads(jsonObj)
            optList.append(optDict)
        
        opt_df = pd.DataFrame(optList)
        opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
        opt_best_df = pd.concat([opt_best_df,opt_df.sort_values('target',ascending=False).iloc[0:2]])

opt_best_df.sort_values('target',ascending=False)
# opt_df = pd.DataFrame(optList)
# opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
# opt_df.sort_values('target',ascending=False)

,target,datetime,bagging_fraction,bagging_freq,batch_size,learning_rate,nFile,num_iterations,num_leaves
24,0.737572,"{'datetime': '2022-12-14 09:47:13', 'elapsed':...",0.833968,1.357303,10.0,0.087423,40.0,664.099012,47.169789
21,0.737376,"{'datetime': '2022-12-14 09:09:26', 'elapsed':...",0.766158,3.669568,10.0,0.092895,40.0,653.033071,47.138080
24,0.735923,"{'datetime': '2022-12-14 05:08:57', 'elapsed':...",0.900000,1.892784,10.0,0.100000,30.0,647.701054,47.199136
24,0.735431,"{'datetime': '2022-12-13 02:41:31', 'elapsed':...",0.755812,8.255519,10.0,0.070067,10.0,972.180853,18.477186
3,0.735058,"{'datetime': '2022-12-12 21:24:28', 'elapsed':...",0.787855,1.994299,10.0,0.094274,10.0,479.436874,23.859578
23,0.734861,"{'datetime': '2022-12-14 04:53:31', 'elapsed':...",0.801450,1.083993,10.0,0.079315,30.0,652.272230,39.776200
10,0.733835,"{'datetime': '2022-12-14 12:59:27', 'elapsed':...",0.608767,1.805601,10.0,0.082465,50.0,660.058584,45.575792
12,0.733727,"{'datetime': '2022-12-14 13:37:39', 'elapsed':...",0.872912,2.097131,10.0,0.054549,50.0,673.196402,49.448597
24,0.732564,"{'datetime': '2022-12-15 00:02:39', 'elapsed':...",0.900000,1.000000,10.0,0.100000,60.0,653.771650,50.000000
19,0.732370,"{'datetime': '2022-12-13 19:30:41', 'elapsed':...",0.900000,1.000000,10.0,0.100000,20.0,484.258199,25.485740


In [3]:
opt_best_df.mean()

target                0.732588
bagging_fraction      0.788105
bagging_freq          3.356691
batch_size           10.000000
learning_rate         0.084519
nFile                40.000000
num_iterations      668.340689
num_leaves           40.620792
dtype: float64

In [4]:
opt_best_df.median()

target                0.733781
bagging_fraction      0.794653
bagging_freq          2.030233
batch_size           10.000000
learning_rate         0.086871
nFile                40.000000
num_iterations      656.915117
num_leaves           47.153935
dtype: float64

In [5]:
def get_sim(df_hist, df_lookup, sim_file_list, score_name_list):
    df_hist['ListenYes'] = (df_hist['skip_2'] == False)*1
    df_hist['ListenYes'].replace(0, -1, inplace = True)
    df_hist = df_hist.groupby(['session_id', 'clus']).agg({'ListenYes':['sum']})
    df_hist = df_hist.reset_index()
    df_hist.columns = df_hist.columns.droplevel(level = 1) # take out the unwanted level
    df_pivot = pd.pivot_table(df_hist, values = 'ListenYes',index='session_id', columns='clus')
    df_pivot = df_pivot.fillna(0)
    
    
    for sim_file, score_name in zip(sim_file_list, score_name_list):
        sim_matrix = pd.read_csv(sim_file).drop(columns=['Unnamed: 0'])
        sim_matrix.columns = list(map(str, range(0,len(sim_matrix))))
        df_sim_session = df_pivot.dot(sim_matrix)/sim_matrix.sum()
        
        df_lookup[score_name] = df_sim_session.lookup(df_lookup['session_id'],df_lookup['clus'].astype(str))
    
    return df_lookup

In [6]:
file_list = []
for n in range(9):
    file_list = file_list + glob.glob('../data/raw/training_set/log_'+str(n)+'*.csv')

import random
random.Random(23).shuffle(file_list) # randomly shuffle the list

In [7]:
def prep_dfs(file, tf_df):
    log_df = pd.read_csv(file)
    log_df = log_df.merge(kmean300_df)

    log_df_1 = log_df.loc[log_df['session_position']<=(log_df['session_length']/2)]
    log_df_1['hour_of_day'] = log_df_1['hour_of_day'].astype('float')
    log_df_1['premium'] = log_df_1['premium'].astype('bool')
    log_df_1['weekday'] = log_df_1['date'].astype('datetime64[ns]').dt.dayofweek
    log_df_1 = log_df_1.drop(columns = ['date'])
    log_df_1 = pd.get_dummies(log_df_1, columns=['hist_user_behavior_reason_end', 'hist_user_behavior_reason_start', 'context_type','weekday'], dtype = 'bool')
    log_df_1 = log_df_1.merge(tf_df.drop(columns = ['time_signature','mode','key']))
    
                     
    col_bool = log_df_1.columns[log_df_1.dtypes=='bool']
    col_nonbool = log_df_1.columns[log_df_1.dtypes!='bool'].drop(['session_id','track_id_clean','clus'])
    
    # the non-convertable values will be set to 0
    log_df_1[col_nonbool] = log_df_1[col_nonbool].apply(pd.to_numeric, errors='coerce', downcast = 'float').fillna(0).astype('float32')

    # aggregate the track history where ['skip_2']==True
    log_df_1_summary_skip2True = pd.concat([log_df_1.loc[log_df_1['skip_2']==True].groupby(['session_id'])[col_bool].agg(['mean']), 
                                            log_df_1.loc[log_df_1['skip_2']==True].groupby(['session_id'])[col_nonbool].agg(['mean', 'std', 'median'])],
                                            axis = 1)
    log_df_1_summary_skip2True.columns = log_df_1_summary_skip2True.columns.get_level_values(0)+'_sk2True_'+log_df_1_summary_skip2True.columns.get_level_values(1)
    
    # aggregate the track history where ['skip_2']==False
    log_df_1_summary_skip2False = pd.concat([log_df_1.loc[log_df_1['skip_2']==False].groupby(['session_id'])[col_bool].agg(['mean']), 
                                             log_df_1.loc[log_df_1['skip_2']==False].groupby(['session_id'])[col_nonbool].agg(['mean', 'std', 'median'])],
                                             axis = 1)
    log_df_1_summary_skip2False.columns = log_df_1_summary_skip2False.columns.get_level_values(0)+'_sk2False_'+log_df_1_summary_skip2False.columns.get_level_values(1)
    
    
    log_df_history = log_df_1[['session_id','track_id_clean','skip_2','clus']]


    half_cut = log_df['session_length']/2

    # need to at least include 2 trials, otherwise the log_df_1_summary will confound with all the tracks in the same session

    #1st trial in the 2nd half
    log_df_2_1 = log_df.loc[(log_df['session_position']>half_cut) & (log_df['session_position']<=half_cut+1)]
    log_df_2_1 = log_df_2_1[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2_1['weight'] = 1

    #2nd trial in the 2nd half
    log_df_2_2 = log_df.loc[(log_df['session_position']>half_cut+1) & (log_df['session_position']<=half_cut+2)]
    log_df_2_2 = log_df_2_2[['session_id','track_id_clean','skip_2','session_position','session_length','clus']]
    log_df_2_2['weight'] = 0.75

    log_df_2 = pd.concat([log_df_2_1,log_df_2_2])
    log_df_2 = log_df_2.merge(log_df_1_summary_skip2True, on='session_id')
    log_df_2 = log_df_2.merge(log_df_1_summary_skip2False, on='session_id')

    sim_file_list = ['../models/SVD/all_tracks/similarity/k300_CanbDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_CosSim.csv',
                     '../models/SVD/all_tracks/similarity/k300_LinCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_ManhDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_HammDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_SpearCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_KendCorr.csv',
                     '../models/SVD/all_tracks/similarity/k300_ChebDist.csv',
                     '../models/SVD/all_tracks/similarity/k300_BrayDist.csv']
    score_name_list = ['CanbDist300', 'CosSim300','LinCorr300','ManhDist300','HammDist300','SpearCorr300','KendCorr300','ChebDist','BrayDist']

    return get_sim(log_df_history, log_df_2, sim_file_list, score_name_list)

In [ ]:
import random
from timeit import default_timer as timer #to see how long the computation will take

nFile = 0
batch_size = 10
while nFile < len(file_list):
    start = timer()
    nFile += batch_size
    df_lookup_list = []
    for file in file_list[(nFile-batch_size):min(nFile, len(file_list))]:
        df_lookup_list.append(prep_dfs(file, tf_df))

    df_lookup = pd.concat(df_lookup_list)
    df_lookup = df_lookup.merge(tf_df.drop(columns = ['key','time_signature','mode']))
    
    # check whether the column names match with the previous training set
    if nFile>batch_size:
        prev_feature_names = lgb.Booster(model_file='../models/SVD/LightGBM_BayesOpt_dec12/LightGBM_incremental_training_dec16/boost'+str(int(nFile-batch_size))+'.txt').feature_name()
        if bool(set(prev_feature_names) - set(df_lookup.columns)): # if there are missing columns
            df_lookup[list(set(prev_feature_names) - set(df_lookup.columns))] = 0 # add the missed columns with 0

    dtrain = lgb.Dataset(df_lookup.drop(columns = ['session_id','track_id_clean','skip_2','weight']).astype('float32'), 
                     label=df_lookup['skip_2'],
                     weight = df_lookup['weight'],
                     free_raw_data=False) # https://lightgbm.readthedocs.io/en/latest/FAQ.html#error-messages-cannot-before-construct-dataset

    
    params = {'num_leaves': 24,
              'learning_rate':0.09,
              'metric': 'binary_error',
              'num_iterations':480,
              'bagging_fraction':0.8,
              'bagging_freq':2,
              'objective': 'binary',
              'force_row_wise': True,
              'num_threads': 5,
              'verbosity': 0,
              'tree_learner': 'voting'} #https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html
    
    if nFile == batch_size:
        bst = lgb.train(params, dtrain)
    else: # continue training on the previous model
        bst = lgb.train(params, dtrain, init_model='../models/SVD/LightGBM_BayesOpt_dec12/LightGBM_incremental_training_dec16/boost'+str(int(nFile-batch_size))+'.txt')
        
    bst.save_model('../models/SVD/LightGBM_BayesOpt_dec12/LightGBM_incremental_training_dec16/boost'+str(int(nFile))+'.txt')

    print('Runtime per batch: %0.2fs' % (timer() - start))

Runtime per batch: 739.76s
Runtime per batch: 631.04s
Runtime per batch: 605.05s
Runtime per batch: 716.47s
Runtime per batch: 696.40s
Runtime per batch: 682.61s
Runtime per batch: 756.11s
Runtime per batch: 744.21s
Runtime per batch: 730.16s
Runtime per batch: 727.42s
Runtime per batch: 787.79s
Runtime per batch: 776.63s
Runtime per batch: 883.42s
Runtime per batch: 897.19s
Runtime per batch: 993.23s
Runtime per batch: 1133.09s
Runtime per batch: 1028.33s
Runtime per batch: 1174.07s
Runtime per batch: 1281.22s
Runtime per batch: 1363.78s
Runtime per batch: 1415.34s
Runtime per batch: 1455.43s
Runtime per batch: 1697.62s
Runtime per batch: 1612.20s
Runtime per batch: 1679.12s
Runtime per batch: 1768.61s
Runtime per batch: 1833.88s
Runtime per batch: 1861.30s
Runtime per batch: 2001.38s
Runtime per batch: 2133.81s
Runtime per batch: 2385.80s
Runtime per batch: 2178.07s
Runtime per batch: 2294.76s
Runtime per batch: 2272.99s
Runtime per batch: 2340.99s
Runtime per batch: 2467.31s
Runtime